# Support Vector Machine

서포트 벡터 머신은 도로의 폭과 마진 오류를 적절하게 조율하는 것이 중요하다. 

모든 샘플이 도로 밖에 위치한다면 이를 하드 마진분류라고 한다. 하드마진분류는 데이터가 선형적으로 분류되어야 하고, 이상치가 없어야한다는 단점이 있다. 그러므로 보통 소프트 마진 오류가 적절한 선택일 것이다. 

서포트 벡터 머신은 피처의 스케일에 민감하게 반응하기 때문에 StandardScaler로 스케일링 해주는게 적절하다. 

SVM모델을 만들 때 여러 하이퍼파라미터를 지정할 수 있다. 그중 C라는 하이퍼파라미터가 있는데, C가 높을수록 과대적합되는 경향이 있다. 
이말은 즉슨 마진 오류가 감소하고 도로의 폭도 함께 함소하는 형태이다.

# 선형 SVM 분류

In [2]:
import numpy as np
from sklearn import datasets 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris['data'][:, (2,3)]
y = (iris['target']==2).astype(np.float64)

# 파이프라인을 만든다. 
# 손실은 힌지 손실을 사용한다.
svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    # 힌지 손실이 적절하다.
    
    ('linear_svc', LinearSVC(C=1, loss='hinge'))
])

svm_clf.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [3]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

---------
> LinearSVC()

대신에 

> SVC(kernel="linear", C=1), SGDClassifier(loss="hinge", alpha=1/(m\*C)) (m은 샘플 수이다.)

를 사용할 수 있다.

# 비선형 SVM 분류

선형분류만으로 모든 문제를 해결하면 좋겠지만 데이터가 비선형적이라면 선형분류는 좋지 않은 모델이다. 비선형 데이터를 다루는 방법은 다항 특성과 같은 특성을 더 추가하는 것이다. 

In [4]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

# 초승달 모양 두개로 이루어진 데이터를 만듦
# ^ 모양과 v 모양이라고 생각하면 될듯..
X, y = make_moons(n_samples=100, noise=0.15)
polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ('scaler', StandardScaler()),
    ('linear_svc', LinearSVC(C=10, loss='hinge'))
])

polynomial_svm_clf.fit(X, y)

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=10, loss='hinge'))])

# 다항식 커널
다항식 특성을 추가하는 것은 간단하고 모든 머신러닝 알고리즘에서 잘 작동하지만 낮은 차수의 다항식은 매우 복잡한 데이터셋을 잘 표현하지 못하고 높은 차수의 다항식은 굉장히 많은 특성을 추가하므로 모델을 너무 느리게 만든다.

다행히도 SVM을 사용할 땐 커널 트릭이라는 거의 기적에 가까운 수학적 기교를 적용할 수 있다. 커널 트릭은 실제로는 특성을 추가하지 않으면서 다항식 특성을 많이 추가한 것과 같은 결과를얻을 수 있다. 사실 어떤 특성도 추가하지 않기 때문에 엄청난 수의 특성 조합이 생기지 않는다. 이 기법은 SVC 파이썬 클래스에 구현되어 있다. 

In [5]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    # coef0은 모델이 높은 차수와 낮은 차수에 얼마나 영향을 받을지 조절할 수 있다.
    # 다항식에 있는 상수 r을 조정하면 고차항의 영향을 줄일 수 있다.
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])

poly_kernel_svm_clf.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

# 유사도 특성
유사도 측정 방식을 통해서 비선형 특성을 다룰 수 있다. 

이부분을 간단하게 쓰자면 몇개의 데이터를(모두 선택할 수도 있음.) 선택해서 랜드마크로 지정하고, 랜드마크와의 거리로 나머지 데이터들을 스케일링(?) 하는 것이라고 할 수 있다. 즉, 랜드마크 개수 만큼의 피처를 가진 데이터를 생성하는 것이다. (원본 데이터의 피처는 제외하고)

# 가우시안 RBF 커널
다항 특성방식과 마찬가지로 유사도 특성 방식도 머신러닝 알고리즘에 유용하게 사용될 수 있다. 하지만 이는 시간이 오래 걸릴 가능성이 너무 높다. 여기서도 마찬가지로 커널트릭을 통해서 시간을 대폭 줄일 수 있다.

In [7]:
# 감마는 가우시안 방사기저함수(RBF)에 포함되는 변수이다. 
# 감마가 클수록 종모양의 그래프가 좁아진다.(표준편차 그래프의 분산이 작아지는 것과 같음)
# 그래서 샘플의 영향 범위가 작아져서 구분선이 구불구불해진다.
# 감마가 작다면 구분선이 좀더 단순해진다.
# 과대적합일 경우에는 감마를 감수시키고 과소적합일 경우에는 증가시켜야 한다. (하이퍼파라미터 C와 비슷하다.)
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])

rbf_kernel_svm_clf.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

<b>대부분의 경우에는 선형커널을 가장 먼저 시도해봐야한다. (LinearSVC가 SVC(kernel="Linear")보다 훨씬 빠르다.)</b>

훈련세트가 크지 않을 경웨는 RBF 커널도 적절한 선택일 것이다.

# SVM 회귀
SVM을 회귀에 사용하는ㄴ 방법은 목표를 반대로 하는 것이다. 일정한 마진 오류 안에서 두 클래스 간의 도로 폭이 가능한 최대가 되도록 하는 대신, SVM 회귀는 제한된 마진 오류 안에서 도로 안에 가능한 많은 샘플이 들어가도록 학습한다. 도로의 폭은 하이퍼파라미터 epsilon으로 조절한다.

In [9]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X,y)

LinearSVR(epsilon=1.5)

In [11]:
from sklearn.svm import SVR

# 서포트 벡터 머신 다항회귀
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X,y)

SVR(C=100, degree=2, kernel='poly')

# SVM 이론

여기서 부터는 SVM의 이론을 설명한다. 그런데... 나는 이게 너무 재미가 없었다. 뭔가 이거를 내가 배워도 쓸모가 없을 것만 같은 느낌..?? 
SVM 공부는 필요할 때 더 하는 걸로 하고 나중으로 미루겠다. 

일단 나는 딥러닝 공부가 너무 재미있었는데 그 공부를 할 것 같다.